# packages

In [27]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tqdm

ModuleNotFoundError: No module named 'tqdm'

# settings

In [11]:
# default column number pandas
pd.options.display.max_columns = 200

# plot style
sns.set(palette = "colorblind", font_scale = 1.5)

# data

In [16]:
hpi = pd.read_excel("/Users/nicholasmichalak/sandbox/hpi-analysis/data/HPI_AT_BDL_cbsa.xlsx", skiprows = 6)

# better column names
## function
def adjust_colname(x: str, to_replace: dict):
    for s, r in to_replace.items():
        # replace
        x = x.replace(s, r)
    return x.lower()

# replace names
hpi.columns = [adjust_colname(col, {" ": "_", "%": "pct", "(": "", ")": ""}) for col in hpi.columns]

# see it
hpi

,cbsa,name,year,annual_change_pct,hpi,hpi_with_1990_base,hpi_with_2000_base
0,1,Alabama(non CBSA areas),1985,.,100.00,91.76,63.20
1,1,Alabama(non CBSA areas),1986,5.89,105.89,97.16,66.92
2,1,Alabama(non CBSA areas),1987,0.78,106.71,97.92,67.44
3,1,Alabama(non CBSA areas),1988,-0.49,106.19,97.44,67.11
4,1,Alabama(non CBSA areas),1989,5.04,111.55,102.35,70.49
...,...,...,...,...,...,...,...
39669,49780,"Zanesville, OH",2017,3.44,278.12,200.33,119.62
39670,49780,"Zanesville, OH",2018,6.62,296.54,213.60,127.54
39671,49780,"Zanesville, OH",2019,4.51,309.92,223.24,133.30
39672,49780,"Zanesville, OH",2020,3.45,320.61,230.93,137.89


# create 5-year HPI intervals for Boulder County

In [25]:
# interval
interval_length = 5

# container
hpi_change_x_year = {"year_start": [], "year_end": [], "hpi_start": [], "hpi_end": [], "change_frac": []}

for year_start_i in tqdm.tqdm(sorted(hpi.query("name == 'Boulder, CO'")["year"].unique())):
    # year end
    year_end_i = year_start_i + interval_length

    if year_end_i <= hpi.query("name == 'Boulder, CO'")["year"].max():

        # hpi start
        hpi_start_i = hpi.query(f"(name == 'Boulder, CO') & (year == {year_start_i})")["hpi"].squeeze()

        # hpi end
        hpi_end_i = hpi.query(f"(name == 'Boulder, CO') & (year == {year_end_i})")["hpi"].squeeze()

        # change fraction
        change_frac_i = (hpi_end_i - hpi_start_i) / hpi_start_i

        # append
        hpi_change_x_year["year_start"].append(year_start_i)
        hpi_change_x_year["year_end"].append(year_end_i)
        hpi_change_x_year["hpi_start"].append(hpi_start_i)
        hpi_change_x_year["hpi_end"].append(hpi_end_i)
        hpi_change_x_year["change_frac"].append(change_frac_i)

# data frame
hpi_change_x_year = pd.DataFrame(hpi_change_x_year)

# see it
hpi_change_x_year

,cbsa,name,year,annual_change_pct,hpi,hpi_with_1990_base,hpi_with_2000_base
6143,14500,"Boulder, CO",1975,.,100.00,37.68,17.00
6144,14500,"Boulder, CO",1976,10.32,110.32,41.57,18.76
6145,14500,"Boulder, CO",1977,18.15,130.35,49.12,22.17
6146,14500,"Boulder, CO",1978,17.06,152.59,57.50,25.95
6147,14500,"Boulder, CO",1979,15.15,175.72,66.22,29.88
6148,14500,"Boulder, CO",1980,8.16,190.05,71.62,32.32
6149,14500,"Boulder, CO",1981,7.13,203.59,76.72,34.62
6150,14500,"Boulder, CO",1982,14.52,233.16,87.86,39.65
6151,14500,"Boulder, CO",1983,-1.40,229.90,86.64,39.09
6152,14500,"Boulder, CO",1984,3.01,236.83,89.25,40.27
